# K8s Python API - Creating Service

## Install python api package

In [1]:
#!pip install kubernetes

## Imports

In [2]:
from kubernetes import client, config, utils
import yaml

## Yaml Templates for k8s objects

In [3]:
deployment_yaml = """
apiVersion: apps/v1
kind: Deployment
metadata:
  name: '**place-holder**'
spec:
  selector:
    matchLabels:
      run: '**place-holder**'
  replicas: 2
  template:
    metadata:
      labels:
        run: '**place-holder**'
    spec:
      containers:
      - name: my-nginx
        image: nginx
        ports:
        - containerPort: 80
"""

service_yaml = """
apiVersion: v1
kind: Service
metadata:
  name: '**place-holder**'
#  labels:
#    run: '**place-holder**'
spec:
  ports:
  - port: 80
    protocol: TCP
  selector:
    run: '**place-holder**'
"""


## setup for for accessing k8s

In [4]:
# get in-cluster config
config.load_incluster_config()

# Client API Interface
api_client = client.ApiClient()

# V1 Core API Interface
v1_api = client.CoreV1Api()

## Create deployment

In [5]:
# convert yaml template to dictionary
deployment = yaml.load(deployment_yaml, Loader=yaml.SafeLoader)

# update name
deployment['metadata']['name'] = 'jmt-nginx'
deployment['spec']['selector']['matchLabels']['run'] = 'jmt-nginx'
deployment['spec']['template']['metadata']['labels']['run'] =  'jmt-nginx'
                                              

# Pass to api client to create deployment object
#deployment_objects = utils.create_from_dict(api_client, deployment, namespace='kubeflow-user')

In [6]:
# convert template to dictionary
service = yaml.load(service_yaml, Loader=yaml.SafeLoader)

# modify for specific service deployment
service['metadata']['name'] = 'jmt-nginx'
service['metadata']['labels'] = {'run': 'jmt-nginx'}
service['spec']['selector']['run'] = 'jmt-nginx'

# pass to api client to create service object
# service_objects = utils.create_from_dict(api_client, service, namespace='kubeflow-user')

In [7]:
# setup list of k8s resources to instanstiate
k8s_resource_list = {
    'apiVersion': 'v1',
    'kind': 'List',
    'items': [deployment, service]
}

# instantiate the k8s resources
k8s_objects = utils.create_from_dict(api_client, k8s_resource_list, namespace='kubeflow-user')

## show k8s objects

In [8]:
for o in k8s_objects:
    print(o.metadata.name, o.kind)

jmt-nginx Deployment
jmt-nginx Service


In [10]:
!kubectl get deploy

NAME                              READY   UP-TO-DATE   AVAILABLE   AGE
ml-pipeline-visualizationserver   1/1     1            1           44h
ml-pipeline-ui-artifact           1/1     1            1           44h
jmt-nginx                         2/2     2            2           12s


In [11]:
!kubectl describe deploy jmt-nginx

Name:                   jmt-nginx
Namespace:              kubeflow-user
CreationTimestamp:      Fri, 17 Dec 2021 16:25:16 +0000
Labels:                 <none>
Annotations:            deployment.kubernetes.io/revision: 1
Selector:               run=jmt-nginx
Replicas:               2 desired | 2 updated | 2 total | 2 available | 0 unavailable
StrategyType:           RollingUpdate
MinReadySeconds:        0
RollingUpdateStrategy:  25% max unavailable, 25% max surge
Pod Template:
  Labels:  run=jmt-nginx
  Containers:
   my-nginx:
    Image:        nginx
    Port:         80/TCP
    Host Port:    0/TCP
    Environment:  <none>
    Mounts:       <none>
  Volumes:        <none>
Conditions:
  Type           Status  Reason
  ----           ------  ------
  Available      True    MinimumReplicasAvailable
  Progressing    True    NewReplicaSetAvailable
OldReplicaSets:  <none>
NewReplicaSet:   jmt-nginx-5bb4fb5ff4 (2/2 replicas created)
Events:
  Type    Reason             Age   From             

In [12]:
!kubectl get pod -l run=jmt-nginx

NAME                         READY   STATUS    RESTARTS   AGE
jmt-nginx-5bb4fb5ff4-qdvw9   2/2     Running   0          18s
jmt-nginx-5bb4fb5ff4-ffd7w   2/2     Running   0          18s


In [13]:
# find first pod of the deployment
ret = v1_api.list_namespaced_pod('kubeflow-user')
for i in ret.items:
    if i.metadata.labels.get('run', None) == 'jmt-nginx':
        pod_to_list = i.metadata.name
        break

In [14]:
!kubectl describe pod $pod_to_list

Name:         jmt-nginx-5bb4fb5ff4-qdvw9
Namespace:    kubeflow-user
Priority:     0
Node:         k3d-kubeflow-server-0/172.18.0.3
Start Time:   Fri, 17 Dec 2021 16:25:18 +0000
Labels:       istio.io/rev=default
              pod-template-hash=5bb4fb5ff4
              run=jmt-nginx
              security.istio.io/tlsMode=istio
              service.istio.io/canonical-name=jmt-nginx
              service.istio.io/canonical-revision=latest
Annotations:  kubectl.kubernetes.io/default-logs-container: my-nginx
              prometheus.io/path: /stats/prometheus
              prometheus.io/port: 15020
              prometheus.io/scrape: true
              sidecar.istio.io/status:
                {"initContainers":["istio-init"],"containers":["istio-proxy"],"volumes":["istio-envoy","istio-data","istio-podinfo","istio-token","istiod-...
Status:       Running
IP:           10.42.0.151
IPs:
  IP:           10.42.0.151
Controlled By:  ReplicaSet/jmt-nginx-5bb4fb5ff4
Init Containers:
  istio-init

In [15]:
!kubectl logs $pod_to_list

Using deprecated annotation `kubectl.kubernetes.io/default-logs-container` in pod/jmt-nginx-5bb4fb5ff4-qdvw9. Please use `kubectl.kubernetes.io/default-container` instead
/docker-entrypoint.sh: /docker-entrypoint.d/ is not empty, will attempt to perform configuration
/docker-entrypoint.sh: Looking for shell scripts in /docker-entrypoint.d/
/docker-entrypoint.sh: Launching /docker-entrypoint.d/10-listen-on-ipv6-by-default.sh
10-listen-on-ipv6-by-default.sh: info: Getting the checksum of /etc/nginx/conf.d/default.conf
10-listen-on-ipv6-by-default.sh: info: Enabled listen on IPv6 in /etc/nginx/conf.d/default.conf
/docker-entrypoint.sh: Launching /docker-entrypoint.d/20-envsubst-on-templates.sh
/docker-entrypoint.sh: Launching /docker-entrypoint.d/30-tune-worker-processes.sh
/docker-entrypoint.sh: Configuration complete; ready for start up
2021/12/17 16:25:21 [notice] 1#1: using the "epoll" event method
2021/12/17 16:25:21 [notice] 1#1: nginx/1.21.4
2021/12/17 16:25:21 [notice] 1#1: built 

In [16]:
!kubectl get service

NAME                              TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)    AGE
ml-pipeline-visualizationserver   ClusterIP   10.43.247.61   <none>        8888/TCP   44h
ml-pipeline-ui-artifact           ClusterIP   10.43.146.99   <none>        80/TCP     44h
test-notebook                     ClusterIP   10.43.81.242   <none>        80/TCP     43h
vscode-notebook                   ClusterIP   10.43.43.50    <none>        80/TCP     42h
jmt-nginx                         ClusterIP   10.43.222.0    <none>        80/TCP     43s


In [17]:
!kubectl describe service jmt-nginx

Name:              jmt-nginx
Namespace:         kubeflow-user
Labels:            run=jmt-nginx
Annotations:       <none>
Selector:          run=jmt-nginx
Type:              ClusterIP
IP Family Policy:  SingleStack
IP Families:       IPv4
IP:                10.43.222.0
IPs:               10.43.222.0
Port:              <unset>  80/TCP
TargetPort:        80/TCP
Endpoints:         10.42.0.151:80,10.42.0.152:80
Session Affinity:  None
Events:            <none>


In [18]:
!kubectl delete deploy jmt-nginx

deployment.apps "jmt-nginx" deleted


In [19]:
!kubectl delete service jmt-nginx

service "jmt-nginx" deleted
